In [1]:
using ArrheniusModel
using Enzyme

In [4]:
T = [300.0]
dT = [0.0]
# b = [0.0 1.0; 1.2 0.0]
b = [0 0.5 0.5; 0.5 0 0.5; 0.5 0.5 0]
db = zero(b)
K = zero(b)
dK = zero(b)
dK[1,1] = 1.0
# dK[2,1] = 1.0
Enzyme.autodiff(ReverseWithPrimal, ar_matrixT!, Duplicated(b, db), Duplicated(K,dK), Duplicated(T,dT))

((nothing, nothing, nothing), nothing)

In [5]:
display(db)

3×3 Matrix{Float64}:
 0.0  1.54017e-7  1.54017e-7
 0.0  0.0         0.0
 0.0  0.0         0.0

In [9]:
dT

1-element Vector{Float64}:
 2.566943074371834e-10

In [6]:
dK

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

In [2]:
T = 300.0
b = [0 0.5 0.5; 0.5 0 0.5; 0.5 0.5 0]
sens_b, sens_T = sens_arrhenius_rate(b,T)
display(sens_b)
display(sens_T)

dK[1, 1] = 
[1.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
db after autodiff for dK[1, 1] = 
[0.0 1.5401658446231002e-7 1.5401658446231002e-7; 0.0 0.0 0.0; 0.0 0.0 0.0]
dT after autodiff for dK[1, 1] = 
[-5.133886148743668e-10]
dK[1, 2] = 
[0.0 1.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
db after autodiff for dK[1, 2] = 
[0.0 -1.5401658446231002e-7 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
dT after autodiff for dK[1, 2] = 
[2.566943074371834e-10]
dK[1, 3] = 
[0.0 0.0 1.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
db after autodiff for dK[1, 3] = 
[0.0 0.0 -1.5401658446231002e-7; 0.0 0.0 0.0; 0.0 0.0 0.0]
dT after autodiff for dK[1, 3] = 
[2.566943074371834e-10]
dK[2, 1] = 
[0.0 0.0 0.0; 1.0 0.0 0.0; 0.0 0.0 0.0]
db after autodiff for dK[2, 1] = 
[0.0 0.0 0.0; -1.5401658446231002e-7 0.0 0.0; 0.0 0.0 0.0]
dT after autodiff for dK[2, 1] = 
[2.566943074371834e-10]
dK[2, 2] = 
[0.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 0.0]
db after autodiff for dK[2, 2] = 
[0.0 0.0 0.0; 1.5401658446231002e-7 0.0 1.5401658446231002e-7; 0.0 0.0 0.0]
dT after

3×3 Matrix{Matrix{Float64}}:
 [0.0 1.54017e-7 1.54017e-7; 0.0 0.0 0.0; 0.0 0.0 0.0]  …  [0.0 0.0 -1.54017e-7; 0.0 0.0 0.0; 0.0 0.0 0.0]
 [0.0 0.0 0.0; -1.54017e-7 0.0 0.0; 0.0 0.0 0.0]           [0.0 0.0 0.0; 0.0 0.0 -1.54017e-7; 0.0 0.0 0.0]
 [0.0 0.0 0.0; 0.0 0.0 0.0; -1.54017e-7 0.0 0.0]           [0.0 0.0 0.0; 0.0 0.0 0.0; 1.54017e-7 1.54017e-7 0.0]

3×3 Matrix{Vector{Float64}}:
 [-5.13389e-10]  [2.56694e-10]   [2.56694e-10]
 [2.56694e-10]   [-5.13389e-10]  [2.56694e-10]
 [2.56694e-10]   [2.56694e-10]   [-5.13389e-10]

In [ ]:


rng = Xoshiro(0)
G = [-5.92, -5.942, -5.97]
Ea = [0.00 1.00 0.01; 1.00 0.00 1.00; 0.01 1.00 0.00]
pe = PhaseEnergies(G, Ea)
T = 300.0
K = arrhenius_rate(pe.barriers, T)
t= 10
dt = 0.05
datasize = Int(t/0.5+1)
num_steps = floor(Int, t/dt)
num_layers = floor(Int, t/0.5)+1
flow_rate = 0.5
decay_coefficient = 0.00001 * flow_rate
fcoeff = flow_coefficient("exponential", num_layers, decay_coefficient)
n = n_phases(pe)
c0 = zeros(num_layers, n)
c0[1, 1] = 1.0
j = 0
j0 = 0
p = (fcoeff, j0, j, dt, num_steps, num_layers)
tspan = (0.0, (num_steps-1) * dt)
tsteps = range(tspan[1], tspan[2]; length = datasize)

function deposition_rates!(dc, c, p, t)
    # Unpack parameters
    fcoeff, j0, j, dt, num_steps, num_layers = p
    # Calculate deposition rates
    j = floor(Int, t / 0.5) + 1
    f = reverse(fcoeff[j: num_layers+j-1])
    dc .= c .* f * K
    if j != j0
        c[j+1, 1] = 1.0
        j = j0
    end
end

function ODE_calculation(barriers=pe.barriers, T=T, prob=prob)
    K = arrhenius_rate(barriers, T)
    ode_data = Array(solve(prob, Euler(), saveat = 0.5, dt = 0.05))
    return ode_data[:,:,end]
end

println(ODE_calculation(pe.barriers, T))

db = Array(zero(pe.barriers))
dT = 1.0

godeT = Enzyme.autodiff(Forward,
 ODE_calculation,
 Duplicated(pe.barriers, db), 
 Duplicated(T,dT), 
 Const(prob))[1]